## Prepare

In [1]:
# importing packages
import seaborn as sns
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler, RobustScaler, Normalizer
from sklearn.metrics import accuracy_score
from sklearn.metrics import accuracy_score, balanced_accuracy_score
from imblearn.over_sampling import SVMSMOTE

In [2]:
#Data
print(os.getcwd())
data_dir = os.getcwd() + "\\Data set\\Parkinson disease.csv"
print(data_dir)

F:\Parkinsson_disease
F:\Parkinsson_disease\Data set\Parkinson disease.csv


In [33]:
df_parkinson = pd.read_csv(data_dir, delimiter = ",")
df_parkinson.head()
df_parkinson.columns

Index(['name', 'MDVP:Fo(Hz)', 'MDVP:Fhi(Hz)', 'MDVP:Flo(Hz)', 'MDVP:Jitter(%)',
       'MDVP:Jitter(Abs)', 'MDVP:RAP', 'MDVP:PPQ', 'Jitter:DDP',
       'MDVP:Shimmer', 'MDVP:Shimmer(dB)', 'Shimmer:APQ3', 'Shimmer:APQ5',
       'MDVP:APQ', 'Shimmer:DDA', 'NHR', 'HNR', 'status', 'RPDE', 'DFA',
       'spread1', 'spread2', 'D2', 'PPE'],
      dtype='object')

In [34]:
#dropna drops missing values
df_parkinson_miss = df_parkinson.dropna(axis=0)
# print(df_parkinson_miss)
df_parkinson = df_parkinson.drop(['name'], axis=1).head()
df_parkinson.head()

,MDVP:Fo(Hz),MDVP:Fhi(Hz),MDVP:Flo(Hz),MDVP:Jitter(%),MDVP:Jitter(Abs),MDVP:RAP,MDVP:PPQ,Jitter:DDP,MDVP:Shimmer,MDVP:Shimmer(dB),...,Shimmer:DDA,NHR,HNR,status,RPDE,DFA,spread1,spread2,D2,PPE
0,119.992,157.302,74.997,0.00784,0.00007,0.00370,0.00554,0.01109,0.04374,0.426,...,0.06545,0.02211,21.033,1,0.414783,0.815285,-4.813031,0.266482,2.301442,0.284654
1,122.400,148.650,113.819,0.00968,0.00008,0.00465,0.00696,0.01394,0.06134,0.626,...,0.09403,0.01929,19.085,1,0.458359,0.819521,-4.075192,0.335590,2.486855,0.368674
2,116.682,131.111,111.555,0.01050,0.00009,0.00544,0.00781,0.01633,0.05233,0.482,...,0.08270,0.01309,20.651,1,0.429895,0.825288,-4.443179,0.311173,2.342259,0.332634
3,116.676,137.871,111.366,0.00997,0.00009,0.00502,0.00698,0.01505,0.05492,0.517,...,0.08771,0.01353,20.644,1,0.434969,0.819235,-4.117501,0.334147,2.405554,0.368975
4,116.014,141.781,110.655,0.01284,0.00011,0.00655,0.00908,0.01966,0.06425,0.584,...,0.10470,0.01767,19.649,1,0.417356,0.823484,-3.747787,0.234513,2.332180,0.410335


In [35]:
# sns.pairplot(df_parkinson.drop(["name"]), hue='status', height=1.2)
# plt.show()

## Training model 

In [36]:
from sklearn.tree import DecisionTreeRegressor

In [39]:
X = df_parkinson.drop(['status'], axis=1)
y = df_parkinson.status
# smote = SVMSMOTE()
# X_smote, y_smote = smote.fit_resample(X, y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2) # 80% training and 20% test
y_test.head()

0    1
Name: status, dtype: int64

In [25]:
# Scale 
scaler = StandardScaler()
X_scale = scaler.fit_transform(X_train)
print(X_scale)

[[-0.42780456 -0.43110916  0.58831741 -0.17852669  0.         -0.15443264
  -0.28960771 -0.15728868  0.15181958  0.25749956  0.18393108  0.08389234
   0.1361668   0.18370642 -0.84141174  0.29204648  1.2722142  -0.40930984
   0.41356389  1.23001873  1.58967726  0.42833676]
 [-0.85367917 -0.02446618  0.54296286  1.4352421   1.41421356  1.34577015
   1.34308005  1.34551091  1.42792473  1.42715859  1.39798345  1.44610457
   1.43773362  1.39817028  0.29326077 -1.64842929 -0.81837976  0.68587751
   1.35233147 -1.34631498 -0.34803343  1.32193381]
 [ 1.7054284   1.58972964 -1.73165395 -1.37620178 -1.41421356 -1.44872524
  -1.40916503 -1.44819678 -1.37731821 -1.33114181 -1.40763581 -1.36968779
  -1.3682416  -1.40746283  1.51015592  1.05068474 -1.12378476 -1.42742941
  -1.35250667 -0.51966131 -1.15978352 -1.3934475 ]
 [-0.42394467 -1.1341543   0.60037368  0.11948636  0.          0.25738773
   0.35569269  0.25997455 -0.20242611 -0.35351635 -0.17427872 -0.16030912
  -0.20565883 -0.17441388 -0.9620

In [26]:
from sklearn.tree import DecisionTreeClassifier
DT_model = DecisionTreeRegressor(random_state = 0) 
DT_model.fit(X_train, y_train)
DT_y_pred = DT_model.predict(X_test)
print('Model accuracy score : {:0.2%}'. format(accuracy_score(y_test, DT_y_pred)))

Model accuracy score : 100.00%


In [28]:
from sklearn.ensemble import RandomForestClassifier
rf_classifier = RandomForestClassifier()
rf_classifier.fit(X_train, y_train)
rf_y_pred = rf_classifier.predict(X_test)
print('Model accuracy score with criterion entropy: {:0.2%}'. format(accuracy_score(y_test, rf_y_pred)))

Model accuracy score with criterion entropy: 100.00%


In [30]:

#Create a Gaussian Classifier
rnd = RandomForestClassifier(n_estimators = 600, max_leaf_nodes = 16, n_jobs = -1)
rnd.fit(X_train, y_train.values.ravel())

y_pred = rnd.predict(X_test)
print(y_pred)

[1]
